# Financial calculations for PV

The goal of this notebook is to document and understand the different PV calculation methodologies being used.

### Workflow:

##### Select a panel from the cec database
- hone in on a specific product using area and output
- look up a certain parameter

##### Calculate the pv output per m² for the given orientation (not yet part of this workflow)
- either using the PV-GIS website (no control over panel type)
- or using the PV-CEC tools in Ladybug (Needs grasshopper know-how)
- or (#ToDo:) using PVlib https://pvlib-python.readthedocs.io/en/stable/gallery/adr-pvarray/plot_simulate_system.html

##### Calculate payback time 
- guess the future electricity price
- PV parameters and energy output from the previous steps
- play with installed area, cost parameters (default settings are realistic assumptions for Germany)

In [1]:
"""
To publish as .html, use the following line of code
jupyter nbconvert --to html --TagRemovePreprocessor.remove_cell_tags='{"hide_code"}' my-notebook.ipynb
"""

import pandas as pd
import numpy as np
import numpy_financial as npf
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

from datetime import datetime as dt
from datetime import date

import plotly.express as px
from plotly.offline import plot
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.graph_objects as go

# The following line also toggles cell visibility:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

### Cash Flow

Some PV Websites propose the simple amortisation time as a suitable metric. [Explainer link](https://www.calculator.net/payback-period-calculator.html)

This is also known as just "Cash Flow . It provides a general indication of solvency. The main weakness is that it doesn't account for the time value of money.


### Payback Period
_Unit: years_

$$ Payback Period = \dfrac{Initial Investment}{Cash flow per year} $$


### Net Present Value (aka. Discounted Cash Flow)

The value of all future cashflows over the entire life of a project, starting in the preesnt (t=0)

$$ NPV = \sum_{t=0}^{M-1} \frac{R_t}{(1 + i)^t} $$

where:
- $R_t$ = net cash flow at time t
- $i$ = discoutn rate
- $t$ = time of the cash flow

In [9]:
# Integrate slider-based energy prediction into the roi calculation

"""
Data from https://www.destatis.de/DE/Themen/Wirtschaft/Preise/Publikationen/Energiepreise/energiepreisentwicklung-pdf-5619001.html
"""
        
def round_date(d):
    if float(d.strftime("%j"))-1 !=0:
        return float(d.strftime("%Y"))
    else:
        return 0.5 + float(d.strftime("%Y"))

def import_historic_electricity_prices():
    p = pd.read_csv("german energy prices.csv")
    p.dropna(inplace=True)
    p["Berichtszeitraum"] = p["Berichtszeitraum"].apply(
        lambda x: dt.strptime(x, "%b-%y").date())
    p["Year"] = p['Berichtszeitraum'].apply(
        lambda x: (round_date(x)))
    p.drop("Berichtszeitraum", axis=1, inplace=True)
    p.set_index("Year", inplace=True)
    return p

def return_energy_price(electricity_x2, electricity_x, electricity_c, periods):
    # Import historic electricity prices
    historic_electricity_prices = import_historic_electricity_prices()
    
    # Dataframe with annual electricity prices for the analysis period
    electricity_predict_period = pd.DataFrame()
    start = dt.now().year - 2008 + 1
    end = dt.now().year -2008 + 1 + periods
    electricity_predict_period["Year"] = pd.Series(np.arange(dt.now().year+1, 
                                                             dt.now().year+1+periods, 1)).astype("float64")
    x = pd.Series(np.arange(start,end, 1))
    electricity_predict_period["Price"] = x.apply(lambda x : electricity_x2*x**2 + electricity_x*x + electricity_c)
    electricity_predict_period.set_index("Year", inplace=True)

    # Dataframe with annual electricirty prices for the whole period (2008-2050)
    electricity_predict_full = pd.DataFrame()
    electricity_predict_full["Year"] = np.arange(2008,2050,1)
    prediction_period = pd.Series(np.arange(0,42,1))
    electricity_predict_full["price (full duration)"] = prediction_period.apply(
        lambda x : electricity_x2*x**2 + electricity_x*x + electricity_c)
    electricity_predict_full.set_index("Year", inplace=True)
    
    # Merge and compare the price predictions
    price_qa = pd.concat([historic_electricity_prices, electricity_predict_full, electricity_predict_period], join="outer", axis=1)
    electricity_fig = px.scatter(price_qa)
    electricity_fig.update_layout(title="Guess the future Electricity price", yaxis_title="€ct/kWh")
    electricity_fig.show()

    return electricity_predict_period
    
    
    
def pv_return_on_investment(periods, installed_area_m2, power_per_m2, 
                            power_output_per_m2_pv, invest_per_kwp, 
                            degradation, autarky, feed_in_tariff,
                            interest, annual_maintenance):

    def npv(cashflow, interest, period):
        return cashflow / (1+interest/100)**period
    
    # Predict energy price
    w = interactive(return_energy_price, 
            electricity_x2=widgets.FloatSlider(min=-0.05, max=0.05, step=0.001, value=-0.009, readout_format=".3f", style=style), 
            electricity_x=widgets.FloatSlider(min=-0.10, max=5.5, step=0.001, value=0.84, style=style), 
            electricity_c=22.7, 
            periods=periods,
            __manual=True)

    future_energy_prices = w.value
    
    # Cashflow calculation
    cashflow_table = pd.DataFrame(index=electricity_predict_period.index)
    headers=["Cash Flow", "Cum. Cash Flow", "NPV", "Cum. NPV"]
    values = []

    # Energy output
    output_0 = power_output_per_m2_pv*installed_area_m2
    cashflow_table["Period"] = range(0,len(cashflow_table))
    cashflow_table["Output"] = output_0*(1-degradation/100)**cashflow_table["Period"] 
    cashflow_table.loc[cashflow_table.index[0], "Output"] = 0 # Set output to 0 for the first year
    """cashflow_table["NPV of Output"] = cashflow_table.apply(
        lambda x:npv(x["Output"], interest, x["Period"]))"""
    cashflow_table["Self Consumption"] = cashflow_table["Output"]*autarky
    cashflow_table["Feed-in Tariff"] = feed_in_tariff

    # Negative Cashflow
    installed_capacity = installed_area_m2 * power_per_m2
    print("Installed Capacity: {}kWp".format(installed_capacity/1000))
    print("Annual Output (year 1): {}kWh".format(output_0))
    principal = invest_per_kwp * installed_capacity / 1000
    print("Principal Investment: €{}".format(principal))
    cashflow_table["Principal"] = 0
    cashflow_table.loc[cashflow_table.index[0], "Principal"] = principal*-1
    cashflow_table["Maintenance"] = (annual_maintenance/100 * principal)*-1
    cashflow_table.loc[cashflow_table.index[0], "Maintenance"] = 0
    cashflow_table["Negative Cashflow"] = cashflow_table["Principal"] + cashflow_table["Maintenance"]
    """        cashflow_table["NPV of Negative Cashflow"] = cashflow_table.apply(
        lambda x: npv(x["Negative Cashflow"], interest, x["Period"]), axis=1)"""

    # Positive Cashflow
    cashflow_table["Electricity Price"] = price_qa["Price"]
    cashflow_table["Avoided Energy Purchase"] = cashflow_table["Self Consumption"]*cashflow_table["Electricity Price"]/100
    cashflow_table["Sold to Grid"] = (cashflow_table["Output"] - cashflow_table[
        "Self Consumption"])*cashflow_table["Feed-in Tariff"]/100
    cashflow_table["Positive Cashflow"] = cashflow_table["Avoided Energy Purchase"] + cashflow_table["Sold to Grid"]

    # Net Cashflow
    cashflow_table.loc[cashflow_table.index[0], "Positive Cashflow"] = 0
    cashflow_table["Net Cashflow"] = cashflow_table["Negative Cashflow"] + cashflow_table["Positive Cashflow"]
    cashflow_table["Cum. Net Cashflow"] = cashflow_table["Net Cashflow"].cumsum()
    cashflow_table["NPV of Cashflow"] = cashflow_table.apply(
        lambda x: npv(x["Net Cashflow"], interest, x["Period"]), axis=1)
    cashflow_table["Cum. NPV of Cashflow"] = pd.Series(cashflow_table["NPV of Cashflow"]).cumsum()

    #print(cashflow_table[["Period", "Avoided Energy Purchase", "Sold to Grid", "Net Cashflow"]])
    #print(cashflow_table.round())

    fig = make_subplots(rows=1, cols=4, column_widths=[0.6, 0.1, 0.1, 0.1], 
                        subplot_titles=["Annual","Cumulative","IRR", "LCOE"])
    # Annual Cashflows
    fig.add_trace(
        go.Bar(y=cashflow_table["Net Cashflow"],name="Net Cash Flow"),
        row=1, col=1)
    fig.add_trace(
        go.Scatter(y=cashflow_table["Cum. Net Cashflow"], name="Cumulative Cash Flow"),
        row=1, col=1)
    fig.add_trace(
        go.Bar(y=cashflow_table["NPV of Cashflow"], name="Present Value"),
        row=1, col=1)
    fig.add_trace(
        go.Scatter(y=cashflow_table["Cum. NPV of Cashflow"], name="Present Value"),
        row=1, col=1)
    fig.add_trace(
        go.Bar(y=[cashflow_table["Net Cashflow"].sum()], name="Sum of Cashflow"),
        row=1, col=2)
    fig.add_trace(
        go.Bar(y=[npf.npv(interest/100,cashflow_table["Net Cashflow"])], name = "NPV of Cashflow [€]"),
        row=1, col=2)
    # IRR
    irr=go.Bar(y=[npf.irr(cashflow_table["Net Cashflow"])], name = "Internal Rate of Return [%]")

    fig.add_trace(irr, row=1, col=3)

    # LCOE
    lcoe = go.Bar(y=[npf.npv(interest/100, cashflow_table["Output"]) / 
                  npf.npv(interest/100, cashflow_table["Negative Cashflow"]*-1)], 
               name = "Levelized Cost of Energy [€/kWh]")
    fig.add_trace(lcoe, row=1, col=4)

    fig.update_layout(
        title="Cash Flow")
    fig.show()
    fig.get_subplot(1,4)

style = {'description_width': 'initial'} 
ww = interactive(pv_return_on_investment, 
            periods=15,
            installed_area_m2 = widgets.FloatSlider(min=0, max=1000, step=1, value=10, style=style),
            power_per_m2 = widgets.FloatSlider(min=100, max=300, step=1, value=200, style=style),
            invest_per_kwp = widgets.FloatSlider(min=800, max=2000, step=50, value=1250, style=style), 
            power_output_per_m2_pv = widgets.FloatSlider(min=100, max=1300, step=50, value=800, style=style),
            degradation = widgets.FloatSlider(min=0,max=10, step=0.5, value=2, style=style),
            autarky = widgets.FloatSlider(min=0,max=1, step=0.05, value=0.5, style=style),
            feed_in_tariff= widgets.FloatSlider(min=0,max=15, step=0.5, value=6, style=style), 
            interest = widgets.FloatSlider(min=-20,max=20, step=0.1, value=6, style=style), 
            annual_maintenance = widgets.FloatSlider(min=0,max=5, step=0.5, value=1, style=style), 
            __manual=True)
display(ww)    


style = {'description_width': 'initial'}    




interactive(children=(IntSlider(value=15, description='periods', max=45, min=-15), FloatSlider(value=10.0, des…

In [10]:
"""
In the above script I am stuck trying to feed the output of return_energy_price into pv_return_on_investment. 
This example seems to solve the problem
https://stackoverflow.com/questions/65057860/how-to-return-a-pandas-dataframe-from-an-ipywidgets-output


"""

import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import Layout, AppLayout
from IPython.display import display
import functools
 
data = {'year': ['2000', '2000','2000','2000','2001','2001','2001','2001', '2002',  '2002', '2002', '2002',
                 '2003','2003','2003','2003','2004', '2004','2004','2004', '2005', '2005', '2005', '2005', 
                 '2006', '2006', '2006', '2006', '2006', '2007', '2007', '2007', '2007', '2008', '2008', '2008', '2008',
                 '2009', '2009', '2009', '2009'],
        
        'purpose':['Holiday', 'Business', 'VFR', 'Study', 'Holiday', 'Business', 'VFR', 'Study', 'Holiday', 'Business',
                   'VFR', 'Study', 'Holiday', 'Business', 'VFR', 'Study', 'Holiday', 'Business', 'VFR', 'Study', 'Holiday',
                   'Business', 'VFR', 'Study', 'Holiday', 'Business', 'VFR', 'Study', 'Holiday', 'Business', 'VFR', 'Study',
                   'Holiday', 'Business', 'VFR', 'Study', 'Holiday', 'Business', 'VFR', 'Study', 'Holiday'
                  ], 
                'market':['Belgium', 'Luxembourg', 'France', 'Spain', 'Norway', 'Sweden', 'Germany', 'Austria', 'Denmark',
                  'Portugal', 'Greece', 'Croatia', 'Belgium', 'Luxembourg', 'France', 'Spain', 'Norway', 'Sweden',
                  'Germany', 'Austria', 'Denmark', 'Portugal', 'Greece', 'Croatia', 'Belgium', 'Luxembourg', 'France',
                  'Spain', 'Norway', 'Sweden', 'Germany', 'Austria', 'Denmark', 'Portugal', 'Greece', 'Croatia', 'Belgium',
                  'Luxembourg', 'France', 'Spain', 'Norway'
                 ]} 
 
df_london = pd.DataFrame (data, columns = ['year','purpose', 'market'])
 
output_dataframe = None

# Get our unique values
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

output = widgets.Output()

# Dropdown listbox
dropdown_year = widgets.Dropdown(description='Year',
                                  options = unique_sorted_values_plus_ALL(df_london.year))

        
        
# Function to filter our dropdown listboxe
def common_filtering(year):
    global output_dataframe
    df = df_london.copy()
        
    filters = []
    
    # Evaluate our dropdown listbox and return booleans for our selections
    if year is not ALL:
        filters.append(df['year'] == year)
    
    output.clear_output()
    
    with output:
        if filters:
            df_filter = functools.reduce(lambda x,y: x&y, filters)
            output_dataframe = df.loc[df_filter]
        else:
            output_dataframe = df
        display(output_dataframe)

def dropdown_year_eventhandler(change):
    common_filtering(change.new)

dropdown_year.observe(dropdown_year_eventhandler, names='value')

ui = widgets.HBox([dropdown_year])

display(ui, output)

Output()

In [ ]:
"""
Tristan suggested the following 

This is pretty cool. I'd recommend splitting your plotting methods out 
from your maths-y methods so you can better edit/debug/understand the code too. 
You can also use VSCode to get all the type hints sorted and documentation 
auto-added to each method too. Something like this would be perfect:
src
├─── __init__.py
├─── tests
│    ├─── test_plot.py
│    └─── test_code.py
├─── code
│    ├─── methods1.py
│    └─── methods1.py
└─── plot.py

"""

from numpy import linspace
import plotly.graph_objects as go 
def leaf_plot(sense, spec):
    fig = go.Figure()
    x = linspace(0,1,101)
    x[0] += 1e-16
    x[-1] -= 1e-16
    positive = sense*x/(sense*x + (1-spec)*(1-x)) 
    negative = 1-spec*(1-x)/((1-sense)*x + spec*(1-x))
    fig.add_trace(
        go.Scatter(x=x, y  = positive, name="Positive",marker=dict( color='red'))
    )
    fig.add_trace(
        go.Scatter(x=x, y  = negative, 
                   name="Negative", 
                   mode = 'lines+markers',
                   marker=dict( color='green'))
    )
    fig.update_layout(
    xaxis_title="Base rate",
    yaxis_title="After-test probability",
    )
    return fig
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Interpreting Test Results"),
    dcc.Graph(id='graph'),
    html.Label([
        "sensitivity",
        dcc.Slider(
            id='sensitivity-slider',
            min=0,
            max=1,
            step=0.01,
            value=0.5,
            marks = {i: '{:5.2f}'.format(i) for i in linspace(1e-16,1-1e-16,11)}
        ),
    ]),
    html.Label([
        "specificity",
        dcc.Slider(
            id='specificity-slider',
            min=0,
            max=1,
            step=0.01,
            value=0.5,
            marks = {i: '{:5.2f}'.format(i) for i in linspace(1e-16,1-1e-16,11)}
        ),
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    Input("sensitivity-slider", "value"),
    Input("specificity-slider", "value")
)
def update_figure(sense, spec):
    return leaf_plot(sense, spec)
# Run app and display result inline in the notebook
app.run_server()

### Discounted Payback Period (DPP)
_Unit: years_

Considers the time value of money. The discounted payback period is longer than the payback period.

$$ Discounted Payback Period = \frac{-ln(\frac{investment amount x discount rate}{cash flow per year})}{ln(1 + discount  rate)} $$

Neither PP nor DPP account for risks such as market volatility or alternative investments.

### Internal Rate of Return (%)

_The Discount rate that makes the net present value of a project zero_

$$ 0 = NPV = \sum_{n=0}^{N} \frac{CF_n}{(1 + IRR)^n} $$

Where:
- $CF_0$ = Initial Investment / Outlay
- $CF_1, CF_2, ... CF_n$ = Cash Flows
- $n$ = each period
- $N$ = Holding Period
- $NPV$ = Net present value
- $IRR$ = Internal Rate of Return

The IRR returns a % value which makes it possible to compare investments. This is a derivative function which is either solved 

Assumes all positive cashflows of a project will be rieinvested at the same rate as the project, instead of the company's cost of capital, therefore possibly not accurately reflecting the profitability and cost of a project.

Alternative: Modified Internal Rate of Return for more accuracy
 Should be analysed together with the Net Present Value (NPV) and Payback Period. 

Important factors to consider when running this calculation:
- Risk Tolerance
- Company's investment needs
- Risk aversion


### Levelized Cost of Energy (LCOE)

The "Levelized Cost of Energy" (LCOE) is the price at which electricity must be generated from a
specific source to break even over the lifetime of the project. 

$$ LCOE = \frac{\sum_{t=1}^{n} \frac{I_t + M_t + F_t}{(1+r)^t}}{\sum_{t=1}^{n} \frac{E_t}{(1+r)^t}} $$

where: 
- $I_t$ = Investment expenditures in year t 
- $M_t$ = operations and maintenance expenditures in year t
- $F_t$ = Fuel expenditures in year t
- $E_t$ = Electricity generation in the year t
- $r$ = discount rate
- $n$ = investment period in years

References:
- [PV Electricity Cost Maps 2014 (PV-Gis paper describing the methodology)](https://www.researchgate.net/profile/Arnulf-Jaeger-Waldau/publication/269100308_Cost_Maps_for_Unsubsidised_Photovoltaic_Electricity/links/5480458d0cf2ccc7f8bb65db/Cost-Maps-for-Unsubsidised-Photovoltaic-Electricity.pdf)
- [PVinsights website (used by the previous as investment cost reference)](http://pvinsights.com/Member/Login.php)
- [Detailed LCOE Studies](https://www.lazard.com/media/450784/lazards-levelized-cost-of-energy-version-120-vfinal.pdf)

### Real rate of return
### Discounted rate of return
### Discounted Payback Period (DPP)

### Discount rate
_Unit: %_

The rate at which an investor needs to invest to get the same cash flow over the same period.

### Weighted average cost of capital (WACC)
_Unit: %_

The discount rate used to compute present vlaue of future cashflows. Represents a firm's cost of capital, where each category of capital is proportionally weighted. Used in place of discount rate because it represents the financial opportunity cost better.


In [ ]:
from scipy.optimize import curve_fit

def linear(x,a,b):
    return a*x+b

def exponential(x,b,c):
    return np.exp(b*x) + c

def square(x,a,b,c):
    return a*x**2 + b*x + c

def cubic(x,a,b,c,d):
    return a*x**3 + b*x**2 + c*x +d

popt_lin, pcov_lin = curve_fit(linear, energy_prices.index, energy_prices["Haushalte"])

popt_sq, pcov_sq = curve_fit(square, list(energy_prices.index), list(energy_prices["Haushalte"]))

popt_exp, pcov_exp = curve_fit(exponential, list(energy_prices.index), list(energy_prices["Haushalte"]))

popt_cub, pcov_cub = curve_fit(cubic, list(energy_prices.index), list(energy_prices["Haushalte"]))

fig = make_subplots(rows=1, cols=1)
fig.add_trace(
    go.Scatter(y=energy_prices["Haushalte"], name="Haushalte"))
fig.add_trace(
    go.Scatter(y=linear(energy_prices.index, *popt_lin),name="linear"),
    row=1, col=1)
fig.add_trace(
    go.Scatter(y=exponential(energy_prices.index, *popt_exp),name="exponential"),
    row=1, col=1)
fig.add_trace(
    go.Scatter(y=square(energy_prices.index, *popt_sq),name="square"),
    row=1, col=1)
fig.add_trace(
    go.Scatter(y=cubic(energy_prices.index, *popt_cub),name="cubic"),
    row=1, col=1)
fig.update_layout(title="Testing different curve fits")
fig.show()
